# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [2]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [3]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-21 23:40:30,262 [INFO] Balance for solar in (l0, q): initializing
2025-10-21 23:40:30,265 [INFO] ✔ Completed in 0.0005891323089599609 seconds
2025-10-21 23:40:30,267 [INFO] Binding consume in domain (solar, l0, q)
2025-10-21 23:40:30,269 [INFO] ✔ Completed in 0.0004429817199707031 seconds
2025-10-21 23:40:30,270 [INFO] Balance for wind in (l0, y): initializing
2025-10-21 23:40:30,272 [INFO] ✔ Completed in 0.00022220611572265625 seconds
2025-10-21 23:40:30,273 [INFO] Binding consume in domain (wind, l0, y)
2025-10-21 23:40:30,274 [INFO] ✔ Completed in 0.00015997886657714844 seconds
2025-10-21 23:40:30,276 [INFO] Balance for power in (l0, q): initializing
2025-10-21 23:40:30,278 [INFO] ✔ Completed in 0.0002789497375488281 seconds
2025-10-21 23:40:30,281 [INFO] Binding release in domain (power, l0, q)
2025-10-21 23:40:30,285 [INFO] ✔ Completed in 0.0004799365997314453 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [4]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-21 23:40:30,311 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-21 23:40:30,316 [INFO] ✔ Completed in 0.0007688999176025391 seconds
2025-10-21 23:40:30,321 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-21 23:40:30,323 [INFO] ✔ Completed in 0.0010609626770019531 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [5]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-21 23:40:30,345 [INFO] Binding operate in domain (wf, l0, q)
2025-10-21 23:40:30,350 [INFO] ✔ Completed in 0.0009353160858154297 seconds
2025-10-21 23:40:30,353 [INFO] Mapping operate: (wf, l0, q) → (wf, l0, y)
2025-10-21 23:40:30,357 [INFO] ✔ Completed in 0.00028896331787109375 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [6]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-21 23:40:30,394 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-21 23:40:30,397 [INFO] ✔ Completed in 0.0006327629089355469 seconds
2025-10-21 23:40:30,403 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-21 23:40:30,406 [INFO] ✔ Completed in 0.0006549358367919922 seconds
2025-10-21 23:40:30,408 [INFO] Binding operate in domain (pv, l0, q)
2025-10-21 23:40:30,410 [INFO] ✔ Completed in 0.0007920265197753906 seconds
2025-10-21 23:40:30,415 [INFO] Mapping operate: (pv, l0, q) → (pv, l0, y)
2025-10-21 23:40:30,418 [INFO] ✔ Completed in 0.0002930164337158203 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [7]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.discharge.capacity <= 100

2025-10-21 23:40:30,444 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-21 23:40:30,447 [INFO] ✔ Completed in 0.0003628730773925781 seconds
2025-10-21 23:40:30,448 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-21 23:40:30,451 [INFO] ✔ Completed in 0.001138925552368164 seconds
2025-10-21 23:40:30,459 [INFO] Balance for lii.stored in (l0, y): initializing
2025-10-21 23:40:30,464 [INFO] Binding capacity in domain (lii.charge, l0, y)
2025-10-21 23:40:30,475 [INFO] ✔ Completed in 0.0002970695495605469 seconds
2025-10-21 23:40:30,481 [INFO] Binding capacity in domain (lii.discharge, l0, y)
2025-10-21 23:40:30,484 [INFO] ✔ Completed in 0.00046515464782714844 seconds


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [8]:
m.pem = Process()

m.pem(-m.power) == m.solar 

In [9]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-21 23:40:30,531 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, pv)
2025-10-21 23:40:30,533 [INFO] ✔ Completed in 0.0005822181701660156 seconds
2025-10-21 23:40:30,536 [INFO] Balance for solar in (l0, q): adding expend(solar, l0, q, operate, pv)
2025-10-21 23:40:30,538 [INFO] ✔ Completed in 0.00039124488830566406 seconds
2025-10-21 23:40:30,541 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, wf)
2025-10-21 23:40:30,545 [INFO] ✔ Completed in 0.0005061626434326172 seconds
2025-10-21 23:40:30,553 [INFO] Balance for wind in (l0, y): adding expend(wind, l0, y, operate, wf)
2025-10-21 23:40:30,556 [INFO] ✔ Completed in 0.00032711029052734375 seconds
2025-10-21 23:40:30,557 [INFO] Assuming  lii.stored inventory capacity is unbounded in (l0, y)
2025-10-21 23:40:30,561 [INFO] Assuming inventory of lii.stored is unbounded in (l0, y)
2025-10-21 23:40:30,563 [INFO] Assuming inventory of lii.stored is bound by inventory capacity in (l0, 

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [10]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [11]:
m.usd.spend.opt()

2025-10-21 23:40:30,825 [INFO] Mapping spend: (usd, l0, y, capacity, wf) → (usd, l0, y)
2025-10-21 23:40:30,832 [INFO] ✔ Completed in 0.0005099773406982422 seconds
2025-10-21 23:40:30,834 [INFO] Mapping spend: (usd, l0, y, operate, wf) → (usd, l0, y)
2025-10-21 23:40:30,841 [INFO] ✔ Completed in 0.0003230571746826172 seconds
2025-10-21 23:40:30,845 [INFO] Mapping spend: (usd, l0, y, capacity, pv) → (usd, l0, y)
2025-10-21 23:40:30,851 [INFO] ✔ Completed in 0.0004341602325439453 seconds
2025-10-21 23:40:30,855 [INFO] Mapping spend: (usd, l0, y, operate, pv) → (usd, l0, y)
2025-10-21 23:40:30,863 [INFO] ✔ Completed in 0.00039386749267578125 seconds
2025-10-21 23:40:30,872 [INFO] Mapping spend: (usd, l0, y, invcapacity, lii.stored) → (usd, l0, y)
2025-10-21 23:40:30,880 [INFO] ✔ Completed in 0.0004260540008544922 seconds
2025-10-21 23:40:30,882 [INFO] Mapping spend: (usd, l0, y, inventory, lii.stored) → (usd, l0, y)
2025-10-21 23:40:30,892 [INFO] ✔ Completed in 0.00036406517028808594 seco

Set parameter Username
Academic license - for non-commercial use only - expires 2025-12-16
Read MPS format model from file Program(example2).mps
Reading time = 0.01 seconds
PROGRAM(EXAMPLE2): 85 rows, 78 columns, 198 nonzeros


2025-10-21 23:40:31,008 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6567U CPU @ 3.30GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 85 rows, 78 columns and 198 nonzeros
Model fingerprint: 0x2d658939
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 70 rows and 63 columns
Presolve time: 0.00s
Presolved: 15 rows, 15 columns, 44 nonzeros
Variable types: 15 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.00%     -    0s

Explored 1 

2025-10-21 23:40:31,128 [INFO] Solution found. Use .output() to display it
2025-10-21 23:40:31,133 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [12]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [13]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [14]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [15]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [16]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [17]:
m.show(True)

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Functions

<IPython.core.display.Math object>